## 2. Pre-processing and Training Data Development

This stage comes in three parts:

I. Create dummy or indicator features for categorical variables (categories -- one hot encoding)

II. Standardize the magnitude of numeric features using a scaler (MSRP, Cost)

III. Split your data into testing and training datasets

In [9]:


# Assuming your cleaned and explored product data is in a DataFrame called 'products'

# New category list (replace with your actual list if different)
new_categories = pd.read_csv("../data/processed/sunligh-categories.csv") 

# Separate features and target variable
features = products.drop('target_variable', axis=1)  # Drop target
target = products['target_variable']

# Create dummy features for new categories
encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
encoded_features = pd.DataFrame(encoder.fit_transform(features['Category'].str.split(' - ').tolist()))

# Select relevant features (optional, adjust based on your data)
relevant_features = ['Brand', 'Description', 'StyleName'] + list(encoded_features.columns)
features = features[relevant_features]

# Standardize numeric features (if any)
scaler = StandardScaler()
try:
  # Attempt to fit scaler on numeric columns
  numerical_cols = features.select_dtypes(include=['int64', 'float64'])
  scaled_features = pd.concat([scaler.fit_transform(numerical_cols), features.drop(numerical_cols, axis=1)], axis=1)
except ValueError:
  # If no numeric columns, use the features as is
  scaled_features = features.copy()

# Split data into training and testing sets (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(scaled_features, target, test_size=0.2, random_state=42)

# Now you have:
# - X_train: Training features (scaled)
# - X_test: Testing features (scaled)
# - y_train: Training target variable
# - y_test: Testing target variable

# You can use these for model training and evaluation


NameError: name 'products' is not defined

Resources: 

- [blog post](https://towardsdatascience.com/document-embedding-techniques-fed3e7a6a25d)
- [textcategorizer](https://spacy.io/api/textcategorizer)
- [doc2vec](https://radimrehurek.com/gensim/auto_examples/tutorials/run_doc2vec_lee.html#sphx-glr-auto-examples-tutorials-run-doc2vec-lee-py)
- [vec regression](https://towardsdatascience.com/multi-class-text-classification-with-doc2vec-logistic-regression-9da9947b43f4)

In [10]:
# data manipulation and math
#
# import numpy as np
# import scipy as sp
import pandas as pd
#
# plotting and visualization
#
# import matplotlib as mpl
# import matplotlib.cm as cm
# from matplotlib.colors import ListedColormap
# import matplotlib.pyplot as plt
#import seaborn as sns
#
# modeling
#
from sklearn.preprocessing import StandardScaler, OneHotEncoder
# from sklearn.preprocessing import OneHotEncoder as OHE
# import sklearn.model_selection
from sklearn.model_selection import train_test_split
# from sklearn.model_selection import KFold
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import accuracy_score, f1_score
# from sklearn.metrics import classification_report
# from sklearn.metrics import confusion_matrix
# from sklearn.metrics import ConfusionMatrixDisplay

print("Loaded Libraries")

Loaded Libraries


## Categorical Variables

In general, categorical features need to be transformed or encoded to be used in some machine learning models, as is the case for Logistic Regression. A common transformation is so-called dummy encoding, where each possible value of a feature becomes a new column, and a 1 is placed in that column if the data instance (a row of the data) contained that value, and a 0 is placed in that column otherwise.

For example, suppose we had a column in a hypothetical data set called species, and it contained one of two values, "cat" or "dog". The column might look like this:

In [3]:
dflog = pd.read_csv("../data/processed/products.csv")

In [8]:
categorical_features = ['Brand',
                        'Size',
                        'Color', 
                        'Description']
'ParentCategory'
'Category'
dflog = pd.get_dummies(dflog, columns = categorical_features)
print('The data have ', dflog.shape[0], ' rows and ', dflog.shape[1], ' columns\n')
print('column names: \n')
print('\n'.join(list(dflog.columns)))

KeyError: "['Brand', 'Size', 'ParentCategory', 'Category', 'Color'] not in index"

#### Proportion of classes
When building classification models, it is always a good idea to know right away the number of samples per class, proportionally to the total number of samples. First we get the counts of each class.

In [6]:
class_counts = dflog['Category'].value_counts()
class_counts

KeyError: 'Category'

In [7]:
class_percentages = pd.Series([(x / dflog.shape[0]) * 100.00 for x in class_counts])
class_percentages

NameError: name 'class_counts' is not defined

In [7]:
# fig, ax = plt.subplots()
# ax.bar(class_counts.index, class_counts)
# ax.set_xticks([0, 1])
# ax.set_xticklabels(class_percentages.index.astype(str) + '\n' + ' ' +
#                    class_percentages.round(0).astype(str) + '%')
# ax.set_ylabel('Count')
# ax.set_xlabel('Category')
# ax.set_title('Heart Disease class distribution\nwhere 1 means presence of heart disease',
#               fontsize = 10)
# plt.show()

Are there Imbalanced Multi-Class Classification Problems--IMCP going on here?? 

In [ ]:
def points_plot(ax, Xtr, Xte, ytr, yte, 
                clf, 
                mesh = True, colorscale = cmap_light, cdiscrete = cmap_bold, 
                alpha = 0.1, psize = 10, 
                zfunc = False, predicted = False):
#
# note: this function only works for X.shape = (:, 2)
# it is intended to illustrate the classifier boundary
#
# get the column names if they exist to apply
# to the meshed data generated below
#
    try:
        feature_names = Xtr.columns
    except:
        feature_names = None
#        
    Xtrain = np.array(Xtr)
    Xtest = np.array(Xte)
#
    h = 0.02
#
# create a uniform grid spanning the range of the X values
# note that y here is NOT the target, it is the 2nd
# dimension of the desired plot
#
    X = np.concatenate((Xtrain, Xtest))
    x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5
    y_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 100),
                         np.linspace(y_min, y_max, 100))
#
# predict the target value at each point in the grid
#
# this method uses the probabilities from the classifier
# and applies a function to determine the class
#
    if zfunc:
        mesh_data = np.c_[xx.ravel(), yy.ravel()]
        if feature_names is not None:
            mesh_data = pd.DataFrame(mesh_data, 
                         columns = feature_names)
        p0 = clf.predict_proba(mesh_data)[:, 0]
        p1 = clf.predict_proba(mesh_data)[:, 1]
        Z = zfunc(p0, p1)
#
# this method uses the classifier to predict the classes directly
#
    else:
        mesh_data = np.c_[xx.ravel(), yy.ravel()]
        if feature_names is not None:
            mesh_data = pd.DataFrame(mesh_data, 
                                     columns = feature_names)
        Z = clf.predict(mesh_data)
    ZZ = Z.reshape(xx.shape)
#
# plt.pcolormesh() creates a shaded result over the grid
#
    if mesh:
        plt.pcolormesh(xx, yy, ZZ, 
                       cmap = cmap_light, 
                       alpha = alpha, 
                       axes = ax, 
                       shading = 'auto')
#
# add the points to the plot
# these can be the original target values
# or the predicted values
#
    if predicted:
        showtr = clf.predict(Xtr)
        showte = clf.predict(Xte)
    else:
        showtr = ytr
        showte = yte
#
# plot training points
#
    ax.scatter(Xtrain[:, 0], Xtrain[:, 1], 
               c = showtr - 1, 
               cmap = cmap_bold, 
               s = psize, 
               alpha = alpha, 
               edgecolor = "k")
#    
# plot testing points
#
    ax.scatter(Xtest[:, 0], Xtest[:, 1],
               c = showte - 1, 
               cmap = cmap_bold, 
               s = psize + 10,
               alpha = alpha, 
               marker = "s")
    ax.set_xlim(xx.min(), xx.max())
    ax.set_ylim(yy.min(), yy.max())
#
    return ax, xx, yy

In [ ]:
def points_plot_prob(ax, Xtr, Xte, ytr, yte, 
                     clf, colorscale = cmap_light, cdiscrete = cmap_bold, 
                     ccolor = cm, 
                     alpha = 0.1, psize = 10):
    try:
        feature_names = Xtr.columns
    except:
        feature_names = None
#        
    Xtrain = np.array(Xtr)
    Xtest = np.array(Xte)
#    
    ax, xx, yy = points_plot(ax, Xtr, Xte, ytr, yte,
                         clf,
                         mesh = False, 
                         colorscale = colorscale, cdiscrete = cdiscrete, 
                         psize = psize, alpha = alpha,
                         predicted = True) 
    mesh_data = np.c_[xx.ravel(), yy.ravel()]
    if feature_names is not None:
        mesh_data = pd.DataFrame(mesh_data, 
                     columns = feature_names)    
    Z = clf.predict_proba(mesh_data)[:, 1]
    Z = Z.reshape(xx.shape)
    plt.contourf(xx, yy, Z, cmap = ccolor, alpha = 0.2)
    cs2 = plt.contour(xx, yy, Z, cmap = ccolor, alpha = 0.6)
    plt.clabel(cs2, fmt = '%2.1f', colors = 'k', fontsize = 12)
#
    plt.show()

In [1]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import OneHotEncoder

# Create a OneHotEncoder object with sparse_output argument
encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder on the Brand feature
encoded_brand = encoder.fit_transform(df[['Brand']])

# Combine encoded Brand with numerical features (assuming you have some)
combined_features = pd.concat([encoded_brand, df[['Category']]], axis=1)

# Use combined_features for KMeans clustering
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
kmeans.fit(combined_features)

NameError: name 'df' is not defined

NameError: name 'sales' is not defined